In [327]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pandas import Series
%matplotlib inline

In [328]:
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [329]:
properati_no_price= pd.read_csv('../tp1/properati_dataset_testing_noprice_barrios_adivinados.csv',encoding='UTF-8')

In [398]:
def select_columns_and_fill_nan(datos_barrio):
    X_columns = ['created_on','property_type','lat','lon', 
                 'surface_total_in_m2','surface_covered_in_m2',
                 'floor','rooms','expenses']
    datos_barrio = datos_barrio.loc[:,X_columns]
    #relleno los valores de nan con cero
    datos_barrio = datos_barrio.fillna(0)
    return  datos_barrio


propertytype = {}
propertytype[0] = 0
propertytype["appartment"] = 1
propertytype["apartment"] = 1
propertytype["departamento"] = 1
propertytype["house"] = 2
propertytype["casa"] = 2
propertytype["PH"] = 3
propertytype["ph"] = 3
propertytype["store"] = 4
propertytype["local"] = 4

def date_to_float(dt64):
    return (dt64 - np.datetime64('2013-01-01T00:00:00Z')) / np.timedelta64(1, 's')

import re

def convertir_a_float(datos_barrio):
    datos_barrio["expenses"] = datos_barrio["expenses"].map(lambda exp: re.sub("[^0-9]", "",str(exp) ))
    datos_barrio["expenses"] = datos_barrio["expenses"].map(lambda exp: -1 if(str(exp)== "") else exp)
    datos_barrio["property_type"] = datos_barrio.property_type.map(lambda t: propertytype.get(t))
    datos_barrio["created_on"] =  pd.to_datetime(datos_barrio['created_on'])
    datos_barrio['created_on'] = datos_barrio['created_on'].map(lambda dt64 :date_to_float(dt64))
    return datos_barrio


In [384]:
properati_filtered= pd.read_csv('../tp1/properati_filtered1.csv',encoding='UTF-8')


def get_training_set(place):
    properati_filtered_place = properati_filtered.loc[properati_filtered.place_name.str.contains(place, na=False),:]
    
    #si no existe ese barrio uso todo el dataset de prueba
    if(properati_filtered_place.shape[0] == 0):
        properati_filtered_place = properati_filtered
    
    y = properati_filtered_place.price_aprox_usd
    
    properati_filtered_place = select_columns_and_fill_nan(properati_filtered_place)
    properati_filtered_place = convertir_a_float(properati_filtered_place)
    X = properati_filtered_place.loc[:,['created_on','property_type','lat','lon', 'surface_total_in_m2','surface_covered_in_m2','floor','rooms','expenses']]
    
    return (X,y)
    
    

In [334]:
properati_filtered= pd.read_csv('../tp1/properati_filtered_barrios_adivinados.csv',encoding='UTF-8')
def predecir(df_to_predict, X_train, y_train):
    regr = LinearRegression()
    regr.fit(X_train, y_train)
    y_price_usd_predicted = regr.predict(df_to_predict)
    return y_price_usd_predicted
    

In [400]:
def predict_for_place(place):
    df_place_to_predict = properati_no_price.loc[properati_no_price.place_name.str.contains(place, na=False),:]
    X_to_predict = select_columns_and_fill_nan(df_place_to_predict)
    X_to_predict = convertir_a_float(X_to_predict)
    X_train, y_train = get_training_set(place)
    predictions = predecir(X_to_predict, X_train, y_train)
    df_place_to_predict['price_usd'] = predictions
    df_predicted = df_place_to_predict.loc[:,['id','price_usd']]
    return df_predicted

In [402]:
places_to_predict = properati_no_price.place_name.drop_duplicates(keep='first')
df_predictions_list= []
for place in places_to_predict:
    print place
    df_predictions_list.append(predict_for_place(place))

Puerto Madero


/home/jaz/fiuba/orga-de-datos-7506/lib/python2.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/jaz/fiuba/orga-de-datos-7506/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Buenos Aires Interior
Palermo Soho
Chacarita
Colegiales
Almagro
Belgrano
Villa Crespo
Caballito
Boedo
Parque Patricios
Palermo Chico
Palermo
Recoleta
Lomas De Zamora
Flores
La Plata
Temperley
Nuñez
Mataderos
Morón
Capital Federal
Monserrat
Caseros
Boca
Parque Avellaneda
Banfield
Palermo Hollywood
Altos De Hudson I
Villa Urquiza
Barrio Norte
Monte Grande
Villa Luro
Coghlan
El Palomar
Villa Devoto
Barracas
Las Cañitas
Paternal
Wilde
Isidro Casanova
Monte Castro
Berazategui
Ramos Mejía
Retiro
Villa Pueyrredón
Parque Chacabuco
Lanús Este
Moreno
Liniers
Congreso
Centro / Microcentro
Villa Lugano
San Justo
San Cristobal
Pompeya
Adrogué
Avellaneda
Saavedra
Parque Leloir
Villa Luzuriaga
La Tablada
Sáenz Peña
Tribunales
Villa Ortuzar
Balvanera
Once
Valentín Alsina
Crucecita
Turdera
Ciudad Jardín Lomas Del Palomar
Sourigues
Ranelagh
Quilmes
Haedo
Tapiales
Virrey Del Pino
Ituzaingó
San Nicolás
Lomas Del Mirador
San Isidro
Rincon
Tolosa
City Bell
Llavallol
Pilar
La Madrugada
Olivos
Barrio Cerrado


In [404]:

predictions_total = pd.concat(df_predictions_list)

In [405]:
predictions_total.to_csv("prediccion1.csv",encoding='utf-8',index=False)